## 제목을 입력합니다

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from pykrx import stock as st
import pickle

### csv 불러오기 실습

DataFrame : 2차원 데이터 -> table

In [2]:
stock = pd.read_csv("stock.csv")
stock['num'] = range(0, len(stock))
stock_label = stock[['isin','num']]
stock_label.head(10)

,isin,num
0,095570,0
1,006840,1
2,027410,2
3,282330,3
4,138930,4
5,001460,5
6,001465,6
7,001040,7
8,079160,8
9,00104K,9


In [3]:
ohlcv = pd.read_csv('ohlcv최종.csv', dtype={"isin":str})
ohlcv.head(10)

,isin,s_date,open,high,low,close,volume,amount,updown
0,095570,2015-08-21,30900,36500,28000,35500,1994022,64841739950,14.89
1,095570,2015-08-24,33500,34700,31200,32400,392831,12926256800,-8.73
2,095570,2015-08-25,33050,33650,30950,31100,242730,7808534250,-4.01
3,095570,2015-08-26,31300,38800,31300,35450,703721,25144972800,13.99
4,095570,2015-08-27,36200,40300,34800,37850,499913,18764860300,6.77
5,095570,2015-08-28,38300,39200,35450,35700,191315,7088603600,-5.68
6,095570,2015-08-31,36300,36300,33050,33500,203023,6911408900,-6.16
7,095570,2015-09-01,34200,35250,33200,33250,149519,5137381200,-0.75
8,095570,2015-09-02,32600,36650,32000,36100,160592,5569432900,8.57
9,095570,2015-09-03,36150,38950,35300,37900,135862,5091580300,4.99


In [44]:
date = pd.DataFrame(ohlcv['s_date'].unique())
date_label = date.rename({0:'s_date'}, axis = 1)
date_label = date_label.sort_values('s_date')
date_label['num'] = range(0, len(date))
date_label = date_label[['s_date','num']]
date_label.head(10)

,s_date,num
0,2013-01-02,0
1,2013-01-03,1
2,2013-01-04,2
3,2013-01-07,3
4,2013-01-08,4
...,...,...
2584,2023-07-03,2584
2585,2023-07-04,2585
2586,2023-07-05,2586
2587,2023-07-06,2587


In [5]:
ohlcv['s_date'] = ohlcv['s_date'].replace(date_label.set_index('s_date')['num'])
ohlcv['isin'] = ohlcv['isin'].replace(stock_label.set_index('isin')['num'])

In [6]:
ohlcv.head(10)

,isin,s_date,open,high,low,close,volume,amount,updown
0,0,650,30900,36500,28000,35500,1994022,64841739950,14.89
1,0,651,33500,34700,31200,32400,392831,12926256800,-8.73
2,0,652,33050,33650,30950,31100,242730,7808534250,-4.01
3,0,653,31300,38800,31300,35450,703721,25144972800,13.99
4,0,654,36200,40300,34800,37850,499913,18764860300,6.77
5,0,655,38300,39200,35450,35700,191315,7088603600,-5.68
6,0,656,36300,36300,33050,33500,203023,6911408900,-6.16
7,0,657,34200,35250,33200,33250,149519,5137381200,-0.75
8,0,658,32600,36650,32000,36100,160592,5569432900,8.57
9,0,659,36150,38950,35300,37900,135862,5091580300,4.99


In [7]:
grouped_df_list = []

for isin, group in ohlcv.groupby('isin'):
    group['y'] = group['close'].shift(-1)
    group = group.dropna(axis=0, how='any')
    grouped_df_list.append(group)

In [8]:
grouped_df_list[435]

,isin,s_date,open,high,low,close,volume,amount,updown,y
1009708,435,667,114500,116000,106000,113000,41492,4665869000,-1.31,111500.0
1009709,435,668,114500,116500,111000,111500,27335,3090106000,-1.33,109000.0
1009710,435,669,114000,114000,108000,109000,18132,1995071500,-2.24,109500.0
1009711,435,670,110500,111000,108000,109500,9065,988816500,0.46,107500.0
1009712,435,671,107500,109500,107000,107500,8652,935593500,-1.83,106000.0
...,...,...,...,...,...,...,...,...,...,...
1011621,435,2580,96000,96200,95500,96100,475,45504800,0.10,96300.0
1011622,435,2581,96600,96600,95600,96300,379,36357500,0.21,95600.0
1011623,435,2582,96300,96300,94700,95600,1435,136953600,-0.73,96200.0
1011624,435,2583,95200,96600,94600,96200,862,82345100,0.63,95300.0


In [9]:
train = pd.DataFrame([])
test = pd.DataFrame([])
for i, group in enumerate(grouped_df_list):
    if(i % 5 == 0):
        test = pd.concat([test, group])
    else:
        train = pd.concat([train, group])

In [10]:
print(len(train) / (len(test) + len(train)))
print(len(test) / (len(test) + len(train)))

0.8003413200723327
0.19965867992766728


In [11]:
y_train = train.pop("y")
y_train

1936       22292.0
1937       22971.0
1938       23068.0
1939       22971.0
1940       22728.0
            ...   
2212947     2115.0
2212948     2055.0
2212949     2100.0
2212950     2170.0
2212951     2295.0
Name: y, Length: 1770355, dtype: float64

In [12]:
x_train = train
x_train

,isin,s_date,open,high,low,close,volume,amount,updown
1936,1,0,21517,21808,21129,21565,15460,341706450,0.23
1937,1,1,21372,22583,21372,22292,37219,844138300,3.37
1938,1,2,22292,23165,22292,22971,46219,1087385800,3.04
1939,1,3,22971,23746,22486,23068,26358,630975900,0.42
1940,1,4,23068,23165,22244,22971,30637,717271900,-0.42
...,...,...,...,...,...,...,...,...,...
2212947,952,2580,1891,2320,1890,2125,41749319,89510302025,12.73
2212948,952,2581,2090,2115,2055,2115,2042994,4239077890,-0.47
2212949,952,2582,2135,2135,2055,2055,1388206,2915012720,-2.84
2212950,952,2583,2050,2120,2030,2100,1594500,3339019670,2.19


In [13]:
y_test = test.pop("y")
y_test

0          32400.0
1          31100.0
2          35450.0
3          37850.0
4          35700.0
            ...   
2207775     8920.0
2207776     9480.0
2207777     8050.0
2207778     8050.0
2207779     8050.0
Name: y, Length: 441645, dtype: float64

In [14]:
x_test = test
x_test

,isin,s_date,open,high,low,close,volume,amount,updown
0,0,650,30900,36500,28000,35500,1994022,64841739950,14.89
1,0,651,33500,34700,31200,32400,392831,12926256800,-8.73
2,0,652,33050,33650,30950,31100,242730,7808534250,-4.01
3,0,653,31300,38800,31300,35450,703721,25144972800,13.99
4,0,654,36200,40300,34800,37850,499913,18764860300,6.77
...,...,...,...,...,...,...,...,...,...
2207775,950,2580,9090,9090,8090,8800,22177,189178150,-2.22
2207776,950,2581,8610,9450,8500,8920,20045,184586820,1.36
2207777,950,2582,9900,10040,9200,9480,50108,483141270,6.28
2207778,950,2583,8590,8680,8050,8050,30299,255843070,-15.08


In [15]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [16]:
model.score(x_test,y_test)

0.9987625415012088

In [17]:
model.predict(x_test)

array([35501.61470984, 32345.43665662, 31078.34043901, ...,
        9493.64247152,  7973.35682495,  7984.00499914])

In [18]:
pickle.dump(model, open('predict_close_model.sav', 'wb'))

In [19]:
model = pickle.load(open('predict_close_model.sav', 'rb'))

In [20]:
y_test

0          32400.0
1          31100.0
2          35450.0
3          37850.0
4          35700.0
            ...   
2207775     8920.0
2207776     9480.0
2207777     8050.0
2207778     8050.0
2207779     8050.0
Name: y, Length: 441645, dtype: float64

In [21]:
isin = stock_label['isin']
isin

0      095570
1      006840
2      027410
3      282330
4      138930
        ...  
948    005010
949    000540
950    000547
951    000545
952    003280
Name: isin, Length: 953, dtype: object

In [22]:
date_label[date_label['s_date']==max(date_label['s_date'])]

,s_date,num
1935,2023-07-04,2585


In [23]:
df = st.get_market_ohlcv("20230704", "20230707", "005930")
df = df.reset_index()
tmp_map = max(date_label['num'])
df['날짜'] = df['날짜'].astype(str)
for s_date in df['날짜']:
    if s_date not in date_label['s_date'].values:
        new_row = pd.DataFrame({'s_date': [s_date], 'num': [tmp_map + 1]})
        date_label = pd.concat([date_label, new_row], ignore_index=True)
        tmp_map += 1
df['날짜'] = df['날짜'].replace(date_label.set_index('s_date')['num'])
df.tail(10)

,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2585,73400,73600,72900,73000,10214350,747624440200,0.00
1,2586,73000,73300,71900,72000,12310610,889637363400,-1.37
2,2587,71900,72400,71500,71600,14777667,1061491980700,-0.56
3,2588,71100,71400,69800,69900,17308877,1215404338500,-2.37


In [24]:
date_label.tail(10)

,s_date,num
2579,2023-06-26,2579
2580,2023-06-27,2580
2581,2023-06-28,2581
2582,2023-06-29,2582
2583,2023-06-30,2583
2584,2023-07-03,2584
2585,2023-07-04,2585
2586,2023-07-05,2586
2587,2023-07-06,2587
2588,2023-07-07,2588


In [25]:
new_data = pd.DataFrame([])
date_set = ["20230704","20230705","20230706","20230707"]

for date in date_set:
    df = st.get_market_ohlcv(date)
    df = df.reset_index()
    df["s_date"] = date[:4] + "-" + date[4:6] + "-" + date[6:]
    df = df.dropna(axis=0, how='any')
    new_data = pd.concat([new_data,df])
new_data = new_data.rename({"티커":"isin", "시가" : "open", "고가" : "high", "저가" : "low", "종가" : "close", "거래량" : "volume", "거래대금" : "amount", "등락률" : "updown"}, axis=1)
new_data = new_data[['isin', 's_date', 'open', 'high', 'low', 'close', 'volume', 'amount', 'updown']]
new_data = new_data.sort_values(['isin', 's_date'])
new_data['y'] = new_data['close'].shift(-1)
new_data = new_data[new_data["s_date"] != "2023-07-07"]
new_data = new_data.reset_index()
new_data.drop(['index'], axis=1, inplace=True)
new_data['isin'] = new_data['isin'].replace(stock_label.set_index('isin')['num'])
new_data['s_date'] = new_data['s_date'].replace(date_label.set_index('s_date')['num'])
new_data.head(10)

,isin,s_date,open,high,low,close,volume,amount,updown,y
0,340,2585,10300,10440,10260,10390,83550,865107990,0.48,10370.0
1,340,2586,10390,10440,10090,10370,154517,1582256610,-0.19,10300.0
2,340,2587,10390,10390,10190,10300,92202,946743290,-0.68,10100.0
3,84,2585,602,602,577,584,147115,86238821,-1.85,575.0
4,84,2586,598,600,573,575,227654,132339906,-1.54,574.0
5,84,2587,575,579,561,574,275685,156858824,-0.17,554.0
6,188,2585,9930,9960,9750,9760,6692,65645070,-1.71,9700.0
7,188,2586,9790,9820,9700,9700,4838,47064420,-0.61,9700.0
8,188,2587,9700,9800,9570,9700,7140,68799750,0.00,9840.0
9,458,2585,72700,73400,72400,73200,12318,898902200,0.27,72200.0


In [26]:
y_new_data = new_data.pop("y")
today = new_data['close']
x_new_data = new_data
y_new_data.head(10)

0    10370.0
1    10300.0
2    10100.0
3      575.0
4      574.0
5      554.0
6     9700.0
7     9700.0
8     9840.0
9    72200.0
Name: y, dtype: float64

In [27]:
x_new_data.head(10)

,isin,s_date,open,high,low,close,volume,amount,updown
0,340,2585,10300,10440,10260,10390,83550,865107990,0.48
1,340,2586,10390,10440,10090,10370,154517,1582256610,-0.19
2,340,2587,10390,10390,10190,10300,92202,946743290,-0.68
3,84,2585,602,602,577,584,147115,86238821,-1.85
4,84,2586,598,600,573,575,227654,132339906,-1.54
5,84,2587,575,579,561,574,275685,156858824,-0.17
6,188,2585,9930,9960,9750,9760,6692,65645070,-1.71
7,188,2586,9790,9820,9700,9700,4838,47064420,-0.61
8,188,2587,9700,9800,9570,9700,7140,68799750,0.00
9,458,2585,72700,73400,72400,73200,12318,898902200,0.27


In [28]:
model.score(x_new_data,y_new_data)

0.999033285237086

In [29]:
result = pd.concat([today, pd.DataFrame(model.predict(x_new_data)), y_new_data],axis=1)
result[0] = round(result[0])
result['y'] = round(result['y'])
result[0] = result[0].astype(int)
result['y'] = result['y'].astype(int)
result['예상등락율'] = round((result[0] - result['close']) / result['close'] * 100, 2)
result['실제등락율'] = round((result['y'] - result['close']) / result['close'] * 100, 2)
result['등락 일치'] = ((result['예상등락율'] * result['실제등락율']) / abs(result['예상등락율'] * result['실제등락율']) + 1) * 0.5
result = result.fillna(0)
result['등락 일치'] = result['등락 일치'].astype(int)
result['error'] = round((abs((result['y'] - result[0]) / result['y']) * 100), 2)

In [30]:
result

,close,0,y,예상등락율,실제등락율,등락 일치,error
0,10390,10406,10370,0.15,-0.19,0,0.35
1,10370,10376,10300,0.06,-0.68,0,0.74
2,10300,10307,10100,0.07,-1.94,0,2.05
3,584,599,575,2.57,-1.54,0,4.17
4,575,591,574,2.78,-0.17,0,2.96
...,...,...,...,...,...,...,...
2854,2440,2450,2360,0.41,-3.28,0,3.81
2855,2360,2355,2420,-0.21,2.54,0,2.69
2856,9320,9322,9390,0.02,0.75,1,0.72
2857,9390,9398,9050,0.09,-3.62,0,3.85


In [31]:
result['error'].max()

274.48

In [32]:
result['error'].min()

0.0

In [33]:
available = result[(result['error'] <= 5) & (result['등락 일치'] == 1)]

In [34]:
str(round(len(result[result['error'] <= 5])/len(result) * 100,2)) + "%"

'94.33%'

In [35]:
str(round(len(result[result['등락 일치'] == 1])/len(result) * 100,2)) + "%"

'44.32%'

In [36]:
str(round(len(available)/len(result) * 100,2)) + "%"

'42.25%'